### 결과 파일의 용량이 너무 커서, 파일 업로드 및 노트북 파일 열기가 되지 않아 재수정 후 다시 업로드 합니다

- 본 튜토리얼은 범주형 변수 처리에 있어서 좋은 성능을 보이고 있는 CatBoost 방법론에 대한 자료입니다.
- CatBoost의 설정 파라미터를 알아보고, 이에 대해 다양한 설정값에서의 결과를 알아보고, 최적의 파라미터 조합을 찾아보는 것을 진행합니다.
- 특히, CatBoost에서 범주형 변수를 처리하는 방식인 Ordered TS와 관련하여, 이전부터 있었던 다양한 범주형 변수 처리 방법과 Target Statistic 계산 방법을 이용하여 성능을 비교하고, 어떤 방식이 가장 다범주 변수 처리에 있어 성능이 좋은지를 알아보겠습니다.

## Import modules

In [1]:
import math
import random
import time
import string
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from pylab import rcParams
from scipy.io import loadmat
import category_encoders as ce

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import KFold
from pandas.api.types import is_string_dtype, is_numeric_dtype

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Load datasets

- Bank Marketing Data : 포르투칼 은행의 전화 마케팅 데이터, 전화 마케팅의 성공 여부를 고객의 개인정보를 이용해 예측
- 데이터셋 선정 이유는 이진 분류를 목적으로 명확한 평가가 가능하다는 점과, 가장 큰 이유는 데이터셋에 다범주변수를 많이 포함하고 있다는 점에서 CatBoost에 가장 잘 맞는 데이터라고 생각했습니다.

In [2]:
csv = pd.read_csv("Data_bank-full.csv", encoding = "UTF-8-sig", sep = ";")
csv.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


In [3]:
csv = csv.drop(['day', 'month'], axis=1)
csv.head()

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,198,1,-1,0,unknown,no


- 데이터는 다음과 같이 구성되어 있습니다. (불필요한 변수인 day, month 는 제외하겠습니다)
    - 명목형 변수 : job, marital, education, default, housing, loan, contact, poutcome
    - 수치형 변수 : age, balance, duration, campaign, pdays, previous

In [4]:
csv.describe()

,age,balance,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,4918.000000,63.000000,871.000000,275.000000


- 다음으로는 범주형 변수들에 대한 정리입니다.
- 범주형 변수는 총 8개로, {직업, 혼인여부, 교육 수준, 신용카드 유무(default), 주택융자 여부, 개인대출 여부, 연락 유형, 이전 마케팅 캠페인 결과}가 있습니다.
- 각 column별로 {job: 12개, marital: 3개, education: 4개, default: 2개, housing: 2개, loan: 2개, contact: 3개, poutcome: 4개}의 범주로 구성되어있습니다.

In [5]:
print(len(csv['job'].unique()))
csv['job'].value_counts()

12


blue-collar      9732
management       9458
technician       7597
admin.           5171
services         4154
retired          2264
self-employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           938
unknown           288
Name: job, dtype: int64

In [6]:
print(len(csv['marital'].unique()))
csv['marital'].value_counts()

3


married     27214
single      12790
divorced     5207
Name: marital, dtype: int64

In [7]:
print(len(csv['education'].unique()))
csv['education'].value_counts()

4


secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: education, dtype: int64

In [8]:
print(len(csv['default'].unique()))
csv['default'].value_counts()

2


no     44396
yes      815
Name: default, dtype: int64

In [9]:
print(len(csv['housing'].unique()))
csv['housing'].value_counts()

2


yes    25130
no     20081
Name: housing, dtype: int64

In [10]:
print(len(csv['loan'].unique()))
csv['loan'].value_counts()

2


no     37967
yes     7244
Name: loan, dtype: int64

In [11]:
print(len(csv['contact'].unique()))
csv['contact'].value_counts()

3


cellular     29285
unknown      13020
telephone     2906
Name: contact, dtype: int64

In [12]:
print(len(csv['poutcome'].unique()))
csv['poutcome'].value_counts()

4


unknown    36959
failure     4901
other       1840
success     1511
Name: poutcome, dtype: int64

In [13]:
data = pd.DataFrame()

for i, j in enumerate(csv.iloc[:, : -1].dtypes.items()):
    if j[1] == "int64":
        data = pd.concat([data, csv.iloc[:, i].astype(float)], axis = 1, sort = False)
    else:
#         dummies = pd.get_dummies(csv.iloc[:, i])
#         dummies.columns = [j[0] + "_" + k for k in dummies.columns]
        data = pd.concat([data, csv.iloc[:, i]], axis = 1, sort = False)

In [14]:
data.loc[csv.y == "yes", "y"] = 1.0
data.loc[csv.y == "no", "y"] = -1.0
data.y = data.y.astype(float)

In [15]:
data.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,y
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,261.0,1.0,-1.0,0.0,unknown,-1.0
1,44.0,technician,single,secondary,no,29.0,yes,no,unknown,151.0,1.0,-1.0,0.0,unknown,-1.0
2,33.0,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,76.0,1.0,-1.0,0.0,unknown,-1.0
3,47.0,blue-collar,married,unknown,no,1506.0,yes,no,unknown,92.0,1.0,-1.0,0.0,unknown,-1.0
4,33.0,unknown,single,unknown,no,1.0,no,no,unknown,198.0,1.0,-1.0,0.0,unknown,-1.0
5,35.0,management,married,tertiary,no,231.0,yes,no,unknown,139.0,1.0,-1.0,0.0,unknown,-1.0
6,28.0,management,single,tertiary,no,447.0,yes,yes,unknown,217.0,1.0,-1.0,0.0,unknown,-1.0
7,42.0,entrepreneur,divorced,tertiary,yes,2.0,yes,no,unknown,380.0,1.0,-1.0,0.0,unknown,-1.0
8,58.0,retired,married,primary,no,121.0,yes,no,unknown,50.0,1.0,-1.0,0.0,unknown,-1.0
9,43.0,technician,single,secondary,no,593.0,yes,no,unknown,55.0,1.0,-1.0,0.0,unknown,-1.0


In [16]:
# target 설정
target = data['y']
train = data.drop('y', axis = 1)

In [17]:
# Define catboost encoder
cbe_encoder = ce.cat_boost.CatBoostEncoder()
  
# Fit encoder and transform the features
cbe_encoder.fit(train, target)
train_cbe = cbe_encoder.transform(train)

In [18]:
train_cbe = pd.concat([train_cbe, target], axis=1)
train_cbe

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,y
0,58.0,-0.724893,-0.797530,-0.699877,-0.764078,2143.0,-0.845998,-0.746886,-0.918575,261.0,1.0,-1.0,0.0,-0.816769,-1.0
1,44.0,-0.778858,-0.701022,-0.788810,-0.764078,29.0,-0.845998,-0.746886,-0.918575,151.0,1.0,-1.0,0.0,-0.816769,-1.0
2,33.0,-0.834520,-0.797530,-0.788810,-0.764078,2.0,-0.845998,-0.866358,-0.918575,76.0,1.0,-1.0,0.0,-0.816769,-1.0
3,47.0,-0.854492,-0.797530,-0.728615,-0.764078,1506.0,-0.845998,-0.746886,-0.918575,92.0,1.0,-1.0,0.0,-0.816769,-1.0
4,33.0,-0.763896,-0.701022,-0.728615,-0.764078,1.0,-0.665958,-0.746886,-0.918575,198.0,1.0,-1.0,0.0,-0.816769,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51.0,-0.778858,-0.797530,-0.699877,-0.764078,825.0,-0.665958,-0.746886,-0.701624,977.0,3.0,-1.0,0.0,-0.816769,1.0
45207,71.0,-0.544268,-0.761092,-0.827461,-0.764078,1729.0,-0.665958,-0.746886,-0.701624,456.0,2.0,-1.0,0.0,-0.816769,1.0
45208,72.0,-0.544268,-0.797530,-0.788810,-0.764078,5715.0,-0.665958,-0.746886,-0.701624,1127.0,5.0,184.0,3.0,0.293806,1.0
45209,57.0,-0.854492,-0.797530,-0.788810,-0.764078,668.0,-0.665958,-0.746886,-0.731602,508.0,4.0,-1.0,0.0,-0.816769,-1.0


In [19]:
train_data, test_data = train_test_split(train_cbe, train_size = 0.7, random_state=45) 

train_X = train_data.iloc[:, :-1].reset_index(drop = True) # train_X에 종속변수 제거
train_Y = train_data.iloc[:, -1].reset_index(drop = True) # train_Y에 종속변수 따로 저장

test_X = test_data.iloc[:, :-1].reset_index(drop = True) # test_X에 종속변수 제거
test_Y = test_data.iloc[:, -1].reset_index(drop = True) # test_Y에 종속변수 따로 저장

# CatBoost

- 본 순서에서는 CatBoost에서 제공하는 라이브러리를 사용하여 파라미터 변화에 따른 모델의 변화를 보여주고자 합니다.
- 우선 처음은 디폴트 값을 이용하여 비교하겠습니다.
- 출처: https://catboost.ai/en/docs/concepts/python-reference_catboost

In [20]:
# CatBoost 모델(분류) package 불러오기
from catboost import CatBoostClassifier

# CatBoost 모델 생성하기
model = CatBoostClassifier()

# 모델에 데이터 fitting (학습)
model.fit(train_X, train_Y)

Learning rate set to 0.045038
0:	learn: 0.6357369	total: 146ms	remaining: 2m 25s
1:	learn: 0.5805334	total: 154ms	remaining: 1m 17s
2:	learn: 0.5391957	total: 164ms	remaining: 54.5s
3:	learn: 0.5096662	total: 175ms	remaining: 43.5s
4:	learn: 0.4814840	total: 183ms	remaining: 36.5s
5:	learn: 0.4548829	total: 192ms	remaining: 31.9s
6:	learn: 0.4300262	total: 203ms	remaining: 28.8s
7:	learn: 0.4071757	total: 217ms	remaining: 26.9s
8:	learn: 0.3890248	total: 229ms	remaining: 25.3s
9:	learn: 0.3729027	total: 241ms	remaining: 23.9s
10:	learn: 0.3610817	total: 253ms	remaining: 22.7s
11:	learn: 0.3482704	total: 265ms	remaining: 21.9s
12:	learn: 0.3375201	total: 278ms	remaining: 21.1s
13:	learn: 0.3280350	total: 290ms	remaining: 20.5s
14:	learn: 0.3176462	total: 304ms	remaining: 19.9s
15:	learn: 0.3100661	total: 317ms	remaining: 19.5s
16:	learn: 0.3019027	total: 330ms	remaining: 19.1s
17:	learn: 0.2949399	total: 343ms	remaining: 18.7s
18:	learn: 0.2890869	total: 355ms	remaining: 18.3s
19:	learn

172:	learn: 0.2141767	total: 1.48s	remaining: 7.1s
173:	learn: 0.2140449	total: 1.49s	remaining: 7.08s
174:	learn: 0.2139407	total: 1.5s	remaining: 7.08s
175:	learn: 0.2138737	total: 1.51s	remaining: 7.07s
176:	learn: 0.2137928	total: 1.52s	remaining: 7.05s
177:	learn: 0.2137308	total: 1.53s	remaining: 7.05s
178:	learn: 0.2136394	total: 1.53s	remaining: 7.04s
179:	learn: 0.2135885	total: 1.54s	remaining: 7.04s
180:	learn: 0.2135342	total: 1.55s	remaining: 7.02s
181:	learn: 0.2134914	total: 1.56s	remaining: 7s
182:	learn: 0.2134160	total: 1.56s	remaining: 6.99s
183:	learn: 0.2133205	total: 1.57s	remaining: 6.98s
184:	learn: 0.2132774	total: 1.58s	remaining: 6.96s
185:	learn: 0.2131894	total: 1.59s	remaining: 6.94s
186:	learn: 0.2130872	total: 1.59s	remaining: 6.93s
187:	learn: 0.2130155	total: 1.6s	remaining: 6.91s
188:	learn: 0.2129724	total: 1.61s	remaining: 6.9s
189:	learn: 0.2128827	total: 1.61s	remaining: 6.89s
190:	learn: 0.2128046	total: 1.62s	remaining: 6.87s
191:	learn: 0.21274

350:	learn: 0.2023528	total: 2.79s	remaining: 5.16s
351:	learn: 0.2023012	total: 2.8s	remaining: 5.15s
352:	learn: 0.2022814	total: 2.8s	remaining: 5.14s
353:	learn: 0.2022084	total: 2.81s	remaining: 5.13s
354:	learn: 0.2021560	total: 2.82s	remaining: 5.12s
355:	learn: 0.2020991	total: 2.82s	remaining: 5.11s
356:	learn: 0.2020112	total: 2.83s	remaining: 5.1s
357:	learn: 0.2019594	total: 2.84s	remaining: 5.09s
358:	learn: 0.2019157	total: 2.85s	remaining: 5.08s
359:	learn: 0.2018510	total: 2.85s	remaining: 5.08s
360:	learn: 0.2017770	total: 2.86s	remaining: 5.07s
361:	learn: 0.2017382	total: 2.87s	remaining: 5.06s
362:	learn: 0.2016845	total: 2.88s	remaining: 5.05s
363:	learn: 0.2016598	total: 2.88s	remaining: 5.04s
364:	learn: 0.2015949	total: 2.89s	remaining: 5.03s
365:	learn: 0.2015205	total: 2.9s	remaining: 5.02s
366:	learn: 0.2014898	total: 2.91s	remaining: 5.01s
367:	learn: 0.2014135	total: 2.92s	remaining: 5.01s
368:	learn: 0.2013466	total: 2.92s	remaining: 5s
369:	learn: 0.20124

532:	learn: 0.1933476	total: 4.08s	remaining: 3.58s
533:	learn: 0.1933196	total: 4.09s	remaining: 3.57s
534:	learn: 0.1932867	total: 4.09s	remaining: 3.56s
535:	learn: 0.1932239	total: 4.1s	remaining: 3.55s
536:	learn: 0.1931784	total: 4.11s	remaining: 3.54s
537:	learn: 0.1931266	total: 4.11s	remaining: 3.53s
538:	learn: 0.1930739	total: 4.12s	remaining: 3.52s
539:	learn: 0.1930368	total: 4.13s	remaining: 3.52s
540:	learn: 0.1929973	total: 4.13s	remaining: 3.51s
541:	learn: 0.1929831	total: 4.14s	remaining: 3.5s
542:	learn: 0.1929495	total: 4.15s	remaining: 3.49s
543:	learn: 0.1928697	total: 4.15s	remaining: 3.48s
544:	learn: 0.1928382	total: 4.16s	remaining: 3.47s
545:	learn: 0.1927991	total: 4.17s	remaining: 3.46s
546:	learn: 0.1927659	total: 4.17s	remaining: 3.46s
547:	learn: 0.1927069	total: 4.18s	remaining: 3.45s
548:	learn: 0.1926938	total: 4.19s	remaining: 3.44s
549:	learn: 0.1926455	total: 4.2s	remaining: 3.43s
550:	learn: 0.1925895	total: 4.2s	remaining: 3.42s
551:	learn: 0.19

694:	learn: 0.1864107	total: 5.18s	remaining: 2.27s
695:	learn: 0.1863819	total: 5.18s	remaining: 2.26s
696:	learn: 0.1863455	total: 5.19s	remaining: 2.26s
697:	learn: 0.1863126	total: 5.2s	remaining: 2.25s
698:	learn: 0.1862616	total: 5.2s	remaining: 2.24s
699:	learn: 0.1862086	total: 5.21s	remaining: 2.23s
700:	learn: 0.1861797	total: 5.22s	remaining: 2.22s
701:	learn: 0.1861208	total: 5.22s	remaining: 2.22s
702:	learn: 0.1860897	total: 5.23s	remaining: 2.21s
703:	learn: 0.1860620	total: 5.24s	remaining: 2.2s
704:	learn: 0.1860114	total: 5.25s	remaining: 2.19s
705:	learn: 0.1859803	total: 5.25s	remaining: 2.19s
706:	learn: 0.1859445	total: 5.26s	remaining: 2.18s
707:	learn: 0.1859022	total: 5.26s	remaining: 2.17s
708:	learn: 0.1858749	total: 5.27s	remaining: 2.16s
709:	learn: 0.1858329	total: 5.28s	remaining: 2.15s
710:	learn: 0.1857791	total: 5.28s	remaining: 2.15s
711:	learn: 0.1857516	total: 5.29s	remaining: 2.14s
712:	learn: 0.1856963	total: 5.3s	remaining: 2.13s
713:	learn: 0.18

861:	learn: 0.1801044	total: 6.28s	remaining: 1s
862:	learn: 0.1800657	total: 6.28s	remaining: 997ms
863:	learn: 0.1800323	total: 6.29s	remaining: 990ms
864:	learn: 0.1799861	total: 6.3s	remaining: 983ms
865:	learn: 0.1799345	total: 6.3s	remaining: 975ms
866:	learn: 0.1799035	total: 6.31s	remaining: 968ms
867:	learn: 0.1798806	total: 6.32s	remaining: 961ms
868:	learn: 0.1798401	total: 6.32s	remaining: 953ms
869:	learn: 0.1798207	total: 6.33s	remaining: 946ms
870:	learn: 0.1797788	total: 6.33s	remaining: 938ms
871:	learn: 0.1797381	total: 6.34s	remaining: 931ms
872:	learn: 0.1797148	total: 6.35s	remaining: 923ms
873:	learn: 0.1796710	total: 6.35s	remaining: 916ms
874:	learn: 0.1796466	total: 6.36s	remaining: 909ms
875:	learn: 0.1796123	total: 6.37s	remaining: 901ms
876:	learn: 0.1795788	total: 6.37s	remaining: 894ms
877:	learn: 0.1795461	total: 6.38s	remaining: 887ms
878:	learn: 0.1795142	total: 6.39s	remaining: 879ms
879:	learn: 0.1794665	total: 6.39s	remaining: 872ms
880:	learn: 0.179

In [21]:
# 학습한 CatBoost 모델을 통해 테스트 데이터 예측
# pred 변수에 실제값 추가
pred = pd.DataFrame(test_Y)

# 생성된 모델로 예측하기 / pred 변수에 예측값 추가
pred["pred"] = model.predict(test_X)

pred.head(10)

,y,pred
0,-1.0,-1.0
1,-1.0,-1.0
2,-1.0,-1.0
3,-1.0,-1.0
4,-1.0,-1.0
5,-1.0,-1.0
6,-1.0,-1.0
7,-1.0,1.0
8,-1.0,1.0
9,-1.0,-1.0


In [22]:
# Confusion Matrix 생성
tab = pd.crosstab(pred.y, pred.pred)

# 정확도 구하기
acc = (tab.iloc[0,0] + tab.iloc[1,1]) / len(test_Y)

print("Confusion Matrix")
print(tab)
print("   ")
print("Acc : ", acc)

Confusion Matrix
pred   -1.0   1.0
y                
-1.0  11639   362
 1.0    907   656
   
Acc :  0.9064435269831908


- 실험 결과, Ordered TS를 이용한 디폴트 상태의 CatBoost 분류기는 {0.8978178, 89.78%}의 정확도를 보이고있습니다.
- 이를 기준으로, 다양한 hyperparameter에 대한 실험에 있어 baseline으로 설정하여 실험을 진행하겠습니다.

## Hyperparameter (iteration, learning_rate, depth, bootstrap_type, grow_policy)

- 아래는 파라미터에 대한 설명입니다.
- 출처: https://catboost.ai/en/docs/concepts/python-reference_catboost

    Iterations (int): 구축할 수 있는 최대 트리 수
    learning_rate: 기울기 단계를 줄이는 데 사용 (0.0 ~ 1.0)
    depth (int): 나무의 깊이를 정의. 의사결정 트리에서 max_depth 매개변수와 유사함. default: 6이고, 값을 줄이면 과적합이 방지되지만, 학습이 잘 진행되지 않을 수 있음. 2 또는 3을 권장
    bootstrap_type (string): 트리 구조를 구축할 때 트리 분할을 선택하는 알고리즘 측면의 정규화 및 속도에 영향을 미칩니다.
        - 'Poisson'
        - 'Bayesian'
        - 'Bernoulli'
        - 'MVS'
        - 'No'
    grow_policy: 트리 구성에 있어서 어떤 형식을 따를지 선택
        - SymmetricTree — 트리는 지정된 깊이에 도달할 때까지 레벨별로 구성됩니다. 각 반복에서 마지막 트리 수준의 모든 리프는 동일한 조건으로 분할됩니다. 결과 트리 구조는 항상 대칭입니다.
        - Depthwise — 지정된 깊이에 도달할 때까지 트리가 레벨별로 구성됩니다. 각 반복에서 마지막 트리 수준의 모든 비단말 리프가 분할됩니다. 각 리프는 손실 개선이 가장 좋은 상태로 분할됩니다.
        - Lossguide — 지정된 최대 리프 수에 도달할 때까지 리프별로 트리가 생성됩니다. 각 반복에서 손실 개선이 가장 좋은 비단말 리프가 분할됩니다.

In [23]:
iterations_list = ['default', 50, 100, 500, 1000]

In [24]:
learning_rates_list = ['auto', 0.01, 0.05, 0.1]

In [25]:
depths_list = [2, 3, 6] # default: 6

In [26]:
bootstrap_types_list = ['Poisson', 'Bayesian', 'Bernoulli', 'MVS', 'No']

In [27]:
grow_policys_list = ['SymmetricTree', 'Depthwise', 'Lossguide']

### AUROC & ACC 최종 비교 (Hyperparameters)

In [ ]:
parameters = {'iterations': iterations_list,
          'learning_rate': learning_rates_list,
          'depth': depths_list,
          'bootstrap_type': bootstrap_types_list,
          'grow_policy': grow_policys_list,
         }

# model define
model = CatBoostClassifier()

grid_model_clf = GridSearchCV(model, param_grid=parameters, cv=5, scoring='accuracy', refit=True, return_train_score=True)
grid_model_clf.fit(train_X, train_Y)

- 출력 결과가 너무 많아서 학습과정은 지우도록 하겠습니다.

In [29]:
# Results from Grid Search
print("\n========================================================")
print(" Results from Grid Search ")
print("========================================================")
print("\n The best estimator across ALL searched params:\n",
      grid_model_clf.best_estimator_)
print("\n The best score across ALL searched params:\n",
      grid_model_clf.best_score_)
print("\n The best parameters across ALL searched params:\n",
      grid_model_clf.best_params_)
print("\n ========================================================")


 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.9019179748079068

 The best parameters across ALL searched params:
 {'bootstrap_type': 'No', 'depth': 6, 'grow_policy': 'Lossguide', 'iterations': 500, 'learning_rate': 0.01}



In [30]:
# print results
result = pd.DataFrame(grid_model_clf.cv_results_)

best_model_clf = grid_model_clf.best_estimator_

In [31]:
print("CV score")
result[["params"] + ["split" + str(i) + "_test_score" for i in range(5)] + ["std_test_score", "mean_test_score"]]

CV score


,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_test_score,mean_test_score
0,"{'bootstrap_type': 'Poisson', 'depth': 2, 'gro...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'bootstrap_type': 'Poisson', 'depth': 2, 'gro...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'bootstrap_type': 'Poisson', 'depth': 2, 'gro...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'bootstrap_type': 'Poisson', 'depth': 2, 'gro...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'bootstrap_type': 'Poisson', 'depth': 2, 'gro...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
895,"{'bootstrap_type': 'No', 'depth': 6, 'grow_pol...",0.900474,0.901738,0.895402,0.888766,0.899826,0.004745,0.897241
896,"{'bootstrap_type': 'No', 'depth': 6, 'grow_pol...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
897,"{'bootstrap_type': 'No', 'depth': 6, 'grow_pol...",0.902686,0.903002,0.899668,0.897298,0.906304,0.003077,0.901792
898,"{'bootstrap_type': 'No', 'depth': 6, 'grow_pol...",0.896998,0.899842,0.891768,0.893348,0.899194,0.003182,0.896230


- 최종적으로, 데이터셋에 대해 모든 하이퍼파라미터에 대한 gridsearchCV를 진행해서 총 1080개의 경우를 조사했고, 최적의 파라미터를 찾는 과정을 진행했습니다.

In [32]:
print("CatBoost by gridsearchCV")
print("Best Parameter : " + str(grid_model_clf.best_params_))

# predict
pred = best_model_clf.predict(test_X)
# cross table
tab = pd.crosstab(test_Y.ravel(), pred, rownames=["real"], colnames=["pred"])
print(tab)
print("Acc : " + str((tab.iloc[0, 0] + tab.iloc[1, 1]) / len(test_X)))

CatBoost by gridsearchCV
Best Parameter : {'bootstrap_type': 'No', 'depth': 6, 'grow_policy': 'Lossguide', 'iterations': 500, 'learning_rate': 0.01}
pred   -1.0   1.0
real             
-1.0  11672   329
 1.0    914   649
Acc : 0.9083603656738425


- 결과, Best Parameter : {'bootstrap_type': 'No', 'depth': 6, 'grow_policy': 'Lossguide', 'iterations': 500, 'learning_rate': 0.01}일 때 accuracy의 값이 가장 좋게 나왔고,
- 이 모델을 사용해 testset을 분류한 결과, 가장 높은 정확도인 90.84%의 정확도를 달성할 수 있었습니다.